## From Molecular Hamiltonian to Qubit Hamiltonian 

In these notes we'll look at how to use openfermion and openfermionsci4 to find the hamiltonian for a molecule, convert that hamiltonian into qubit operations, and, if possible, compute some properties of that hamiltonian. Be sure you have numpy, openfermion, and openfermionpsi4 installed.  Installation instructions can be found at the [OpenFermion Site](https://github.com/quantumlib/OpenFermion).  

First things first, import the libraries. 

In [4]:
import numpy as np
import openfermion as of 
from openfermionpsi4 import run_psi4

With that done we can specify the properties of the molecule as an openfermion `MolecularData` object and use the psi4 plugin to calculate the one and two electron intregrals along with other useful properties of the molecule. This is fairly well documented in both the OpenFermion paper and their tutorials so I won't belabor it here. 

In [5]:
R = 0.7414
multiplicity = 1
charge = 0 
basis = 'sto-3g'
molecule = of.MolecularData([('H', (0., 0., 0.)),
                             ('H', (0., 0., R))],
                            basis,multiplicity, charge)

# Compute interaction terms
molecule = run_psi4(molecule,
                    run_mp2=True,
                    run_cisd=True,
                    run_ccsd=True,
                    run_fci=True)

Now that psi4 has done its thing we can get the hamiltonian as an openfermion `InteractionOperator`. Their notation for the ladder operators is discussed in the `FermionOperator` section of the paper. 

In [6]:
hamiltonian = molecule.get_molecular_hamiltonian()
print(hamiltonian)

() 0.7137539905449153
((0, 1), (0, 0)) -1.2524635715927759
((1, 1), (1, 0)) -1.2524635715927759
((2, 1), (2, 0)) -0.47594871726831145
((3, 1), (3, 0)) -0.47594871726831145
((0, 1), (0, 1), (0, 0), (0, 0)) 0.3372443828669513
((0, 1), (0, 1), (2, 0), (2, 0)) 0.09064440419713088
((0, 1), (1, 1), (1, 0), (0, 0)) 0.3372443828669513
((0, 1), (1, 1), (3, 0), (2, 0)) 0.09064440419713088
((0, 1), (2, 1), (0, 0), (2, 0)) 0.09064440419713078
((0, 1), (2, 1), (2, 0), (0, 0)) 0.33173404792821903
((0, 1), (3, 1), (1, 0), (2, 0)) 0.09064440419713078
((0, 1), (3, 1), (3, 0), (0, 0)) 0.33173404792821903
((1, 1), (0, 1), (0, 0), (1, 0)) 0.3372443828669513
((1, 1), (0, 1), (2, 0), (3, 0)) 0.09064440419713088
((1, 1), (1, 1), (1, 0), (1, 0)) 0.3372443828669513
((1, 1), (1, 1), (3, 0), (3, 0)) 0.09064440419713088
((1, 1), (2, 1), (0, 0), (3, 0)) 0.09064440419713078
((1, 1), (2, 1), (2, 0), (1, 0)) 0.33173404792821903
((1, 1), (3, 1), (1, 0), (3, 0)) 0.09064440419713078
((1, 1), (3, 1), (3, 0), (1, 0)) 0.33

We can now use `transforms.brvyi_kitaev` to convert the Trasform to qubit operations using the Bravyi-Kitaev transformation. The result is an openfermion `QubitOperator`.

In [7]:
bk = of.transforms.bravyi_kitaev(of.transforms.get_fermion_operator(hamiltonian))
print(bk)

(-0.0988639735178176+0j) [] +
(0.04532220209856541+0j) [X0 Z1 X2] +
(0.04532220209856541+0j) [X0 Z1 X2 Z3] +
(0.04532220209856541+0j) [Y0 Z1 Y2] +
(0.04532220209856541+0j) [Y0 Z1 Y2 Z3] +
(0.17119774853325856+0j) [Z0] +
(0.17119774853325875+0j) [Z0 Z1] +
(0.16586702396410952+0j) [Z0 Z1 Z2] +
(0.16586702396410952+0j) [Z0 Z1 Z2 Z3] +
(0.12054482186554412+0j) [Z0 Z2] +
(0.12054482186554412+0j) [Z0 Z2 Z3] +
(0.16862219143347565+0j) [Z1] +
(-0.2227859289010693+0j) [Z1 Z2 Z3] +
(0.17434844170557143+0j) [Z1 Z3] +
(-0.22278592890106935+0j) [Z2]


For simple molecules like Hydrogen we can do some computations directly.  Openfermion has utility functions for computing the expectation and variance. To use them we need a state vector as a numpy array and the scipy LinearOperator form of a `QubitOperator`. The later we can get using `generate_linear_qubit_operator`.

In [8]:
phi = 1/2*(np.eye(16)[0]+np.eye(16)[1]+np.eye(16)[4]+np.eye(16)[8])
bk_op = of.utils.generate_linear_qubit_operator(bk)
print("Expectation w.r.t phi: " + str(of.utils.expectation(bk_op,phi)))
print("Variance w.r.t. phi: " + str(of.utils.variance(bk_op,phi)))

Expectation w.r.t phi: (-0.17440103348480254+0j)
Variance w.r.t. phi: (0.501929193526253+0j)


But wait! There's more!  We can even get the eigenspectrum. Recall the the goal of quantum simulation is to compute the minimum eigenvalue for a given hamiltonian. Thus, we should not expect this to scale well. If it did we wouldn't really be studying the VQE algorithm.  

In [9]:
eig_spec =  of.utils.eigenspectrum(bk)
print(eig_spec)
print("Ground State Energy: " + str(eig_spec.min()))


[-1.13727017 -0.53870958 -0.53870958 -0.53247901 -0.53247901 -0.53247901
 -0.44698572 -0.44698572 -0.16990139  0.23780527  0.23780527  0.35243413
  0.35243413  0.47983611  0.71375399  0.92010671]
Ground State Energy: -1.137270174625328
